# そもそもどんなデータを扱うの？

ここでは，本サイトで分析対象とするデータについて簡単に紹介します．
なお，このデータは，[MADB Lab v1.0](https://github.com/mediaarts-db/dataset/tree/1.0)を[こちら](https://kakeami.github.io/viz-madb/appendix/preprocess.html)の手順で前処理したものです．

## 環境構築

In [6]:
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [34]:
# 前処理の結果，以下に分析対象ファイルが格納されていることを想定
PATH_DATA = '../../data/preprocess/out/magazines.csv'

In [8]:
df = pd.read_csv(PATH_DATA)

## 概要

`df`は

- 週刊少年ジャンプ
- 週刊少年サンデー
- 週刊少年マガジン

の`1969-11-03`から`2017-07-12`までのすべてのマンガ作品のデータを格納した`DataFrame`です．サイズを見てみます．

In [24]:
df.shape

(135896, 12)

マンガ作品x週を1行に格納するため，約14万行のデータとなっています．列は下記です．

In [10]:
df.columns

Index(['mcname', 'miname', 'cname', 'epname', 'creator', 'pageStart',
       'pageEnd', 'numberOfPages', 'datePublished', 'price', 'publisher',
       'editor'],
      dtype='object')

- `mcname`: 雑誌名
- `miname`: 雑誌巻号名
- `cname`: マンガ作品名
- `epname`: 各話タイトル
- `creator`: 作者名
- `pageStart`: 開始ページ
- `pageEnd`: 終了ページ
- `numberOfPages`: 雑誌の合計ページ数
- `datePublished`: 雑誌の発行日
- `price`: 雑誌の価格
- `publisher`: 雑誌の発行元
- `editor`: 雑誌の編集者（編集長）

冒頭数行を見てみましょう．

In [12]:
df.head(3).T

,0,1,2
mcname,週刊少年ジャンプ,週刊少年ジャンプ,週刊少年ジャンプ
miname,週刊少年ジャンプ 1969年 表示号数20,週刊少年ジャンプ 1969年 表示号数20,週刊少年ジャンプ 1969年 表示号数20
cname,ハレンチ学園,モサ,漫画コント55号
epname,赤い嵐の巻,盗まれた金の巻,ゆうかい魔の巻
creator,永井豪とダイナミックプロ,ちばてつや,榎本有也
pageStart,7.0,39.0,54.0
pageEnd,37.0,53.0,55.0
numberOfPages,296.0,296.0,296.0
datePublished,1969-11-03,1969-11-03,1969-11-03
price,90.0,90.0,90.0


`pandas`の`describe()`コマンドでざっくり集計してみます．

In [21]:
df.describe()

,pageStart,pageEnd,numberOfPages,price
count,135896.000000,135890.000000,135653.000000,135879.000000
mean,213.197213,230.853172,421.200740,202.327107
std,125.282380,123.402589,70.453981,42.839723
min,1.000000,0.000000,36.000000,70.000000
25%,109.000000,128.000000,356.000000,180.000000
50%,207.000000,224.000000,444.000000,210.000000
75%,307.000000,325.000000,474.000000,230.000000
max,2275.000000,1578.000000,600.000000,371.000000


- `pageStart`はすべての行でデータがありますが，それ以外は`NaN`（数値なし）が結構ありそうです
- `numberOfPages`の最小値，最大値が想定外に広がっていたので，あとで深堀りします
- `price`も同様です．後で深堀りします

次に，`NaN`の数を列ごとに集計します．

In [31]:
df.isna().sum()

mcname               0
miname               0
cname                8
epname           22000
creator            378
pageStart            0
pageEnd              6
numberOfPages      243
datePublished        0
price               17
publisher        17896
editor            1992
dtype: int64

- `cname`（マンガ作品名）が不明なものが8行もあります．今の所，補完する手立てはありません．
- `epname`（各話タイトル）がない行が想定より多く，22000行に上ります．こちらも同様に，補完する手立てはありません．
- `creator`（作者）がない行が378行存在します．`cname`が同一のものがあれば補完できる可能性があります．
- `price`（雑誌の販売価格）がない行が17行あります．前後の巻号から推定可能です．
- `publisher`（発行者）がない行が17896行あります．`miname`が同一のものから補完可能です（そもそも`講談社`か`小学館`の2パターンしかありません）
- `editor`（編集長）がない行が1992行あります．こちらも，前後の巻号から推定可能ですが，編集長が交代するタイミングだと打つ手がありません．

## 各列

### `mcname`（雑誌名）

In [36]:
df['mcname'].value_counts()

週刊少年サンデー    46582
週刊少年マガジン    45705
週刊少年ジャンプ    43609
Name: mcname, dtype: int64

上記は`mcname`ごとの行数を表します．それぞれ同一期間で集計しましたが，掲載作品数の違いが生じています．

### `miname`（雑誌巻号名）

ユニークな`miname`数を集計します．

In [38]:
df['miname'].nunique()

7017

`mcname`（雑誌）ごとに集計した`miname`（雑誌巻号）は：

In [40]:
df.groupby('mcname')['miname'].nunique()

mcname
週刊少年サンデー    2341
週刊少年ジャンプ    2338
週刊少年マガジン    2338
Name: miname, dtype: int64

ジャンプとマガジンは同一ですが，サンデーが3号分多いことがわかります．

次は，`miname`（雑誌巻号）ごとに`mcname`（マンガ作品）数を集計してみます．

In [41]:
df.groupby('miname')['cname'].nunique().sort_values().reset_index()

,miname,cname
0,週刊少年マガジン 1970年 表示号数2,7
1,週刊少年サンデー 1969年 表示号数48,8
2,週刊少年マガジン 1970年 表示号数3,8
3,週刊少年マガジン 1969年 表示号数52,8
4,週刊少年マガジン 1971年 表示号数1,8
...,...,...
7012,週刊少年マガジン 2010年 表示号数7,35
7013,週刊少年マガジン 2000年 表示号数1,35
7014,週刊少年ジャンプ 2014年 表示号数4,37
7015,週刊少年ジャンプ 2003年 表示号数37,40


1970年付近の黎明期は掲載数が少なめだったようです．また，雑誌によってタイミングはバラバラですが，非常に沢山のマンガを掲載する**外れ値**が存在することもわかります．

### `cname`（マンガ作品名）

ユニークな`cname`（マンガ作品）数を集計します．

In [43]:
df['cname'].nunique()

5283

`mcname`（雑誌）ごとに`cname`（マンガ作品）数を集計します．

In [47]:
df.groupby('mcname')['cname'].nunique()

mcname
週刊少年サンデー    1524
週刊少年ジャンプ    2185
週刊少年マガジン    1594
Name: cname, dtype: int64

ジャンプが圧倒的に多いですね…．試しに`cname`（マンガ作品）ごとに掲載数を集計してみます．

In [64]:
df_tmp = df[['mcname', 'cname']].value_counts().reset_index()
df_tmp.columns = ['mcname', 'cname', 'counts']
df_tmp

,mcname,cname,counts
0,週刊少年ジャンプ,こちら葛飾区亀有公園前派出所,1968
1,週刊少年マガジン,はじめの一歩,1185
2,週刊少年サンデー,名探偵コナン,1009
3,週刊少年ジャンプ,ONE PIECE,890
4,週刊少年サンデー,MAJOR,748
...,...,...,...
5298,週刊少年ジャンプ,ジャンプ魂,1
5299,週刊少年ジャンプ,ジャン助!フェス太!のレッツゴージャンプフェスタ2002,1
5300,週刊少年ジャンプ,ジュゲムジュゲム,1
5301,週刊少年ジャンプ,スカルジャッカー,1


連載期間が長い方を見てみると，こち亀，はじめの一歩，名探偵コナンと各雑誌のレジェンドが連なり面白いですね．

逆に連載期間が短い方を見てみると，`ジャン助!フェス太!のレッツゴージャンプフェスタ2002`等のマンガもDBに登録されていることがわかります．読み切り作品も登録されているようなので，そのあたりも合計マンガ作品数の差に響いているのかもしれません．

雑誌ごとに，マンガ作品の連載期間に関して基礎集計します．

In [65]:
df_tmp.groupby('mcname')['counts'].describe()

,count,mean,std,min,25%,50%,75%,max
mcname,,,,,,,,
週刊少年サンデー,1524.0,30.563648,73.222929,1.0,1.0,2.0,27.0,1009.0
週刊少年ジャンプ,2185.0,19.957895,73.908870,1.0,1.0,1.0,11.0,1968.0
週刊少年マガジン,1594.0,28.670640,74.795986,1.0,1.0,2.0,20.0,1185.0


やはりジャンプの平均掲載期間が，他誌と比べて短いことがわかります．

### `epname`

### `creator`

### `pageStart`

### `pageEnd`

### `numberOfPages`

### `datePublished`

### `publisher`

### `editor`